In [2]:
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.layers import LSTM, Merge

import pandas as pd
import numpy as np
import os

In [3]:
df_input = pd.read_csv("clean_and_gaussian.csv", index_col=0)
df_input_segmented = pd.read_csv("clean_and_gaussian_segmented.csv", index_col=0)
df_concat = pd.concat([df_input, df_input_segmented])

# Shuffle rows so that not all 10s are at the beginning and all 1s at the end
df_concat = df_concat.sample(frac=1)

y = df_concat.iloc[:,-1]
X = df_concat.iloc[:,0:-1]

X = X.fillna(0)
y = y.fillna(0)

/Users/Shreya/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [4]:
#train test split 
dataset_size = X.shape[0]
train_size = 8*(dataset_size//10)
train_X = X[0:train_size]
train_y = y[0:train_size].to_frame()
test_X = X[train_size:]
test_y = y[train_size:].to_frame()
print(train_X.shape)
print(type(train_X), train_X.shape)
print(type(train_y), train_y.shape)

(5768, 1198)
<class 'pandas.core.frame.DataFrame'> (5768, 1198)
<class 'pandas.core.frame.DataFrame'> (5768, 1)


In [4]:
import tensorflow as tf
from tensorflow.contrib import rnn

#define constants
#unrolled through 28 time steps
time_steps=1198
#hidden LSTM units
num_units=25
#rows of 28 pixels
n_input=1
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 11 classes(0-10).
n_classes=11 
#size of batch
batch_size=25

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

In [5]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [10]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    start_i = 0
    
    while iter<230:
        
        train_X_temp = train_X.iloc[start_i:start_i + batch_size]
        
        batch_x,batch_y=train_X_temp.values.reshape(1,1198,batch_size), pd.get_dummies(train_y['reps'].values).iloc[start_i: start_i + batch_size,:]
        
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))
        batch_y = batch_y.values
        
        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
        start_i = start_i + batch_size
    
    #calculating test accuracy
    test_x_temp = test_X.iloc[0:25].values.reshape(batch_size,time_steps,n_input)
    
    test_y_temp = pd.get_dummies(test_y['reps'].values).iloc[0:25,:]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_x_temp , y: test_y_temp}))


For iter  10
Accuracy  0.12
Loss  2.089879
__________________
For iter  20
Accuracy  0.48
Loss  1.4592088
__________________
For iter  30
Accuracy  0.36
Loss  1.560264
__________________
For iter  40
Accuracy  0.52
Loss  1.2553208
__________________
For iter  50
Accuracy  0.48
Loss  1.3102356
__________________
For iter  60
Accuracy  0.56
Loss  1.1568959
__________________
For iter  70
Accuracy  0.52
Loss  1.1893121
__________________
For iter  80
Accuracy  0.44
Loss  1.4579196
__________________
For iter  90
Accuracy  0.52
Loss  1.2492304
__________________
For iter  100
Accuracy  0.52
Loss  1.2120804
__________________
For iter  110
Accuracy  0.52
Loss  1.0509018
__________________
For iter  120
Accuracy  0.48
Loss  1.1397827
__________________
For iter  130
Accuracy  0.68
Loss  0.9139439
__________________
For iter  140
Accuracy  0.36
Loss  1.2333108
__________________
For iter  150
Accuracy  0.56
Loss  1.213009
__________________
For iter  160
Accuracy  0.48
Loss  1.2324054
_______

In [5]:
import tensorflow as tf
from tensorflow.contrib import rnn

#define constants
#unrolled through 28 time steps
time_steps=1198
#hidden LSTM units
num_units=128
#rows of 28 pixels
n_input=1
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 11 classes(0-10).
n_classes=11 
#size of batch
batch_size=128

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    start_i = 0
    
    while iter<45:
        
        train_X_temp = train_X.iloc[start_i:start_i + batch_size]
        
        batch_x,batch_y=train_X_temp.values.reshape(1,1198,batch_size), pd.get_dummies(train_y['reps'].values).iloc[start_i: start_i + batch_size,:]
        
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))
        batch_y = batch_y.values
        
        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
        start_i = start_i + batch_size
    
    #calculating test accuracy
    test_x_temp = test_X.iloc[0:batch_size].values.reshape(batch_size,time_steps,n_input)
    
    test_y_temp = pd.get_dummies(test_y['reps'].values).iloc[0:batch_size,:]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_x_temp , y: test_y_temp}))


For iter  10
Accuracy  0.5234375
Loss  1.5061315
__________________
For iter  20
Accuracy  0.4765625
Loss  1.3159087
__________________
For iter  30
Accuracy  0.2734375
Loss  1.172572
__________________
For iter  40
Accuracy  0.453125
Loss  1.1294442
__________________
Testing Accuracy: 0.484375
